In [11]:
# Best solution - install everything with compatible versions
!pip install -q torch torchaudio librosa soundfile "numpy>=2" "pandas>=2.2.0" "networkx>=3.0" "gruut[de,es,fr]>=2.4.0" "coqui-tts>=0.26.2"

In [12]:
!pip install git+https://github.com/coqui-ai/TTS.git

  Cloning https://github.com/coqui-ai/TTS.git to /tmp/pip-req-build-gg8f1mkr
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS.git /tmp/pip-req-build-gg8f1mkr
  Resolved https://github.com/coqui-ai/TTS.git to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached gruut-2.2.3-py3-none-any.whl
  Using cached networkx-2.8.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 112.8 M

In [ ]:
# First install all required system dependencies
!apt-get install -qq libportaudio2
!pip install sounddevice



Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1.1) ...
Setting up libportaudio2:amd64 (19.6.0-1.1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero_v2.so.0 is not a symbolic link

/sbin/ldconfig

In [22]:
from IPython.display import display, Audio, clear_output, HTML
import ipywidgets as widgets
from TTS.api import TTS
import torch
import os
import numpy as np
from scipy.io import wavfile
from scipy.signal import lfilter, butter
import io
import sounddevice as sd
from base64 import b64encode

# Custom CSS for styling with premium aesthetics
style = """
<style>
:root {
  --primary: #6c5ce7;
  --secondary: #a29bfe;
  --success: #00b894;
  --danger: #d63031;
  --warning: #fdcb6e;
  --info: #0984e3;
  --light: #f5f6fa;
  --dark: #2d3436;
}

body {
  background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%) !important;
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

.container {
  max-width: 850px;
  margin: 20px auto;
  padding: 30px;
  background: white;
  border-radius: 20px;
  box-shadow: 0 15px 35px rgba(0,0,0,0.1);
  position: relative;
  overflow: hidden;
}

.container::before {
  content: "";
  position: absolute;
  top: 0;
  left: 0;
  right: 0;
  height: 8px;
  background: linear-gradient(90deg, var(--primary) 0%, var(--secondary) 100%);
}

.widget-label {
  font-weight: 600 !important;
  color: var(--dark) !important;
  font-size: 14px !important;
  margin-bottom: 8px !important;
  letter-spacing: 0.3px;
}

/* Updated button styles */
.widget-button {
  background: linear-gradient(135deg, var(--primary) 0%, var(--secondary) 100%) !important;
  color: white !important;
  font-weight: 600 !important;
  border: none !important;
  border-radius: 8px !important;
  box-shadow: 0 2px 5px rgba(0,0,0,0.1) !important;
  padding: 12px 24px !important;
  font-size: 14px !important;
  min-width: 200px !important;
  text-align: center !important;
  margin: 5px !important;
  height: auto !important;
  transition: all 0.2s ease !important;
}

.widget-button:hover {
  transform: translateY(-2px) !important;
  box-shadow: 0 4px 8px rgba(0,0,0,0.15) !important;
}

.upload-button {
  background: linear-gradient(135deg, var(--success) 0%, #55efc4 100%) !important;
}

.record-button {
  background: linear-gradient(135deg, var(--danger) 0%, #ff7675 100%) !important;
}

.button-container {
  display: flex;
  flex-direction: column;
  align-items: center;
  margin: 15px 0;
}

.widget-textarea {
  border: 2px solid #dfe6e9 !important;
  border-radius: 12px !important;
  padding: 12px !important;
  font-size: 14px !important;
  transition: all 0.3s ease !important;
  width: 100% !important;
  margin: 10px 0 !important;
}

.widget-textarea:focus {
  border-color: var(--primary) !important;
  box-shadow: 0 0 0 3px rgba(108, 92, 231, 0.2) !important;
}

.widget-dropdown {
  border: 2px solid #dfe6e9 !important;
  border-radius: 12px !important;
  padding: 10px 12px !important;
  font-size: 14px !important;
  transition: all 0.3s ease !important;
}

.widget-dropdown:focus {
  border-color: var(--primary) !important;
  box-shadow: 0 0 0 3px rgba(108, 92, 231, 0.2) !important;
}

.output_area {
  border-radius: 12px !important;
  background: var(--light) !important;
  padding: 20px !important;
  margin-top: 15px !important;
  border: none !important;
}

.status-success {
  color: var(--success) !important;
  font-weight: 600 !important;
  padding: 10px 15px;
  background: rgba(0, 184, 148, 0.1);
  border-radius: 10px;
  display: inline-block;
  font-size: 14px;
}

.status-error {
  color: var(--danger) !important;
  font-weight: 600 !important;
  padding: 10px 15px;
  background: rgba(214, 48, 49, 0.1);
  border-radius: 10px;
  display: inline-block;
  font-size: 14px;
}

.status-processing {
  color: var(--info) !important;
  font-weight: 600 !important;
  padding: 10px 15px;
  background: rgba(9, 132, 227, 0.1);
  border-radius: 10px;
  display: inline-block;
  font-size: 14px;
}

.header {
  text-align: center;
  margin-bottom: 30px;
}

.header h1 {
  color: var(--primary);
  font-size: 2.5em;
  margin-bottom: 10px;
  font-weight: 700;
  letter-spacing: 0.5px;
}

.header p {
  color: var(--dark);
  font-size: 1.2em;
  opacity: 0.8;
}

.card {
  background: white;
  border-radius: 15px;
  padding: 25px;
  margin-bottom: 25px;
  box-shadow: 0 5px 15px rgba(0,0,0,0.03);
  border: 1px solid rgba(0,0,0,0.05);
  transition: all 0.3s ease;
}

.card:hover {
  transform: translateY(-2px);
  box-shadow: 0 8px 25px rgba(0,0,0,0.08);
}
.feature-icon {
  font-size: 24px;
  margin-right: 10px;
  color: var(--primary);
}

.feature-list {
  display: flex;
  justify-content: space-around;
  flex-wrap: wrap;
  margin: 20px 0;
}

.feature-item {
  display: flex;
  align-items: center;
  margin: 10px;
  padding: 10px 15px;
  background: rgba(108, 92, 231, 0.1);
  border-radius: 30px;
  font-size: 14px;
  font-weight: 500;
  color: var(--dark);
}

.section-header {
  font-size: 16px;
  font-weight: 600;
  color: var(--dark);
  margin: 20px 0 10px 0;
}

.upload-section {
  display: flex;
  flex-direction: column;
  align-items: center;
  margin-bottom: 15px;
}

.or-divider {
  text-align: center;
  margin: 15px 0;
  position: relative;
  color: #636e72;
  font-size: 14px;
}

.or-divider::before, .or-divider::after {
  content: "";
  position: absolute;
  top: 50%;
  width: 45%;
  height: 1px;
  background: #dfe6e9;
}

.or-divider::before {
  left: 0;
}

.or-divider::after {
  right: 0;
}

@media (max-width: 600px) {
  .widget-button {
    width: 100% !important;
  }
}
</style>
"""
display(HTML(style))

# Audio enhancement function
def enhance_audio(input_file, output_file):
    # Read audio file
    sample_rate, data = wavfile.read(input_file)

    # Convert stereo to mono if needed
    if len(data.shape) > 1:
        data = np.mean(data, axis=1)

    # Normalize audio
    data = data / np.max(np.abs(data))

    # Apply noise reduction (simple high-pass filter)
    b, a = butter(4, 100, 'highpass', fs=sample_rate)
    filtered_data = lfilter(b, a, data)

    # Normalize again after filtering
    filtered_data = filtered_data / np.max(np.abs(filtered_data))

    # Write enhanced audio
    wavfile.write(output_file, sample_rate, (filtered_data * 32767).astype(np.int16))

# Initialize TTS model
device = "cuda" if torch.cuda.is_available() else "cpu"
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False).to(device)

# Create widgets with updated styling
upload = widgets.FileUpload(
    accept='.wav,.mp3,.ogg',
    multiple=False,
    description='Select file',
    layout=widgets.Layout(width='auto')
)

upload_btn = widgets.Button(
    description="Upload (0)",
    layout=widgets.Layout(width='auto', margin='5px'),
    style={'button_color': '#00b894'},
    className='upload-button'
)

def update_upload_count(change):
    upload_btn.description = f"Upload ({len(change['new'])})" if change['new'] else "Upload (0)"

upload.observe(update_upload_count, names='value')

text_input = widgets.Textarea(
    value='',
    placeholder='Type what you want the voice to say...',
    rows=4,
    layout=widgets.Layout(width='100%', margin='10px 0')
)

language = widgets.Dropdown(
    options=['English (en)', 'Spanish (es)', 'French (fr)', 'German (de)', 'Italian (it)', 'Portuguese (pt)'],
    value='English (en)',
    layout=widgets.Layout(width='100%', margin='10px 0')
)

generate_btn = widgets.Button(
    description="Generate Voice Clone",
    layout=widgets.Layout(width='auto', margin='20px 0'),
    style={'button_color': '#6c5ce7'}
)

output_audio = widgets.Output(
    layout=widgets.Layout(width='100%', margin='20px 0', padding='20px',
                         border='none', border_radius='12px')
)

status = widgets.HTML(
    value="<div class='status-ready'>Ready to create amazing voice clones!</div>",
    layout=widgets.Layout(width='100%', margin='15px 0')
)

progress = widgets.HTML(
    value="""<div class='progress-container' id='progress-container'>
               <div class='progress-bar' id='progress-bar'></div>
             </div>""",
    layout=widgets.Layout(width='100%')
)

# Microphone recording variables
fs = 44100  # Sample rate
channels = 1  # Mono
recording = False
audio_data = None
stream = None

# Microphone recording widgets
record_btn = widgets.Button(
    description="RECORD VOICE",
    layout=widgets.Layout(width='auto', margin='5px'),
    style={'button_color': '#d63031'},
    className='record-button'
)

stop_btn = widgets.Button(
    description="STOP RECORDING",
    layout=widgets.Layout(width='auto', margin='5px'),
    disabled=True
)

recording_status = widgets.HTML(
    value="<div class='recording-status'><span class='recording-indicator'></span>Recording in progress...</div>",
    layout=widgets.Layout(margin='0 10px', display='none')
)

recorded_audio = widgets.Output(
    layout=widgets.Layout(width='100%', margin='10px 0')
)

def toggle_recording_ui(recording):
    record_btn.disabled = recording
    stop_btn.disabled = not recording
    recording_status.layout.display = 'flex' if recording else 'none'

def start_recording(b):
    global recording, audio_data, stream
    try:
        recording = True
        audio_data = []
        toggle_recording_ui(recording)

        def callback(indata, frames, time, status):
            if recording:
                audio_data.append(indata.copy())

        with recorded_audio:
            clear_output()

        stream = sd.InputStream(callback=callback, channels=channels, samplerate=fs)
        stream.start()
    except Exception as e:
        status.value = f"<div class='status-error'>❌ Error: {str(e)}</div>"
        recording = False
        toggle_recording_ui(recording)

def stop_recording(b):
    global recording, stream
    recording = False
    toggle_recording_ui(recording)

    if stream:
        stream.stop()
        stream.close()

    if audio_data and len(audio_data) > 0:
        audio_array = np.concatenate(audio_data)
        audio_array = (audio_array * 32767).astype(np.int16)  # Convert to 16-bit PCM

        # Save to file
        wavfile.write("recorded_sample.wav", fs, audio_array)

        # Display the recorded audio
        with recorded_audio:
            clear_output()
            display(Audio("recorded_sample.wav", autoplay=False))

            # Create a download link
            with open("recorded_sample.wav", "rb") as f:
                b64 = b64encode(f.read()).decode()
                href = f'<a href="data:audio/wav;base64,{b64}" download="my_recording.wav" style="display: inline-block; padding: 8px 15px; background: #6c5ce7; color: white; border-radius: 4px; text-decoration: none; margin-top: 10px;">⬇️ Download Recording</a>'
                display(HTML(href))

record_btn.on_click(start_recording)
stop_btn.on_click(stop_recording)

def on_generate_click(b):
    # Check if we have either an uploaded file or a recording
    has_upload = bool(upload.value)
    has_recording = os.path.exists("recorded_sample.wav")

    if not has_upload and not has_recording:
        status.value = "<div class='status-error'>⚠️ Please upload a voice sample or record one first</div>"
        return

    if not text_input.value.strip():
        status.value = "<div class='status-error'>⚠️ Please enter text to speak</div>"
        return

    status.value = """<div class='status-processing'>
                      <i class='fas fa-spinner fa-spin'></i> Processing your voice clone...
                      </div>"""
    generate_btn.disabled = True

    # Show progress bar
    display(HTML("""
    <script>
      document.getElementById('progress-container').style.display = 'block';
      let width = 0;
      const progressBar = document.getElementById('progress-bar');
      const interval = setInterval(() => {
        if (width >= 90) {
          clearInterval(interval);
        } else {
          width++;
          progressBar.style.width = width + '%';
        }
      }, 50);
    </script>
    """))

    try:
        # Determine which sample to use (prefer recording if both exist)
        sample_path = "recorded_sample.wav" if has_recording else "uploaded_sample.wav"

        if not has_recording:
            # Save uploaded file if no recording exists
            with open(sample_path, 'wb') as f:
                f.write(next(iter(upload.value.values()))['content'])

        # Get language code from selection
        lang_code = language.value.split('(')[-1].replace(')', '').strip()

        # Generate raw output
        raw_output = "raw_output.wav"
        tts.tts_to_file(
            text=text_input.value,
            speaker_wav=sample_path,
            language=lang_code,
            file_path=raw_output
        )

        # Enhance audio quality
        output_file = "cloned_output.wav"
        enhance_audio(raw_output, output_file)

        # Complete progress bar
        display(HTML("""
        <script>
          document.getElementById('progress-bar').style.width = '100%';
          setTimeout(() => {
            document.getElementById('progress-container').style.display = 'none';
          }, 500);
        </script>
        """))

        # Display audio
        with output_audio:
            clear_output()
            display(HTML("""
            <div style='text-align: center; margin-bottom: 15px;'>
              <h3 style='color: #6c5ce7; margin: 0;'>Your Enhanced Voice Clone</h3>
              <p style='color: #636e72; margin-top: 5px;'>Click play to listen to the results</p>
            </div>
            """))
            display(Audio(output_file, autoplay=False))

            # Add download button
            with open(output_file, "rb") as f:
                b64 = b64encode(f.read()).decode()
                href = f'<a href="data:audio/wav;base64,{b64}" download="cloned_voice.wav" style="display: inline-block; padding: 8px 15px; background: #6c5ce7; color: white; border-radius: 4px; text-decoration: none; margin-top: 10px;">⬇️ Download Clone</a>'
                display(HTML(href))

        status.value = "<div class='status-success'>✅ Success! Your voice clone is ready below</div>"

    except Exception as e:
        status.value = f"<div class='status-error'>❌ Error: {str(e)}</div>"
        display(HTML("<script>document.getElementById('progress-container').style.display = 'none';</script>"))
    finally:
        generate_btn.disabled = False

generate_btn.on_click(on_generate_click)

# Create feature list
features = [
    {"icon": "🎯", "text": "High-Quality Cloning"},
    {"icon": "🔊", "text": "Noise Reduction"},
    {"icon": "🌍", "text": "Multi-Language Support"},
    {"icon": "⚡", "text": "Fast Processing"},
    {"icon": "🎤", "text": "Live Recording"}
]

# Create the app layout with updated structure
app = widgets.VBox([
    widgets.HTML("""
    <div class='header'>
      <h1>✨ AI Voice Cloning Studio</h1>
      <p>Transform any voice sample into a natural-sounding speech</p>
    </div>
    """),
     widgets.HTML("<div class='feature-list'>" +
                 "".join([f"<div class='feature-item'><span class='feature-icon'>{f['icon']}</span>{f['text']}</div>"
                          for f in features]) +
                 "</div>"),

    widgets.HTML("<div class='card'>"),

    # Section 1: Upload
    widgets.HTML("<div class='section-header'>Upload existing recording:</div>"),
    upload_btn,
    upload,  # Hidden file upload that gets triggered by the button

    # Section 2: Record
    widgets.HTML("<div class='section-header'>Record live with microphone:</div>"),
    widgets.HBox([record_btn, stop_btn, recording_status], layout=widgets.Layout(justify_content='center')),
    recorded_audio,

    # Section 3: Text input
    widgets.HTML("<div class='section-header'>Enter Text to Speak:</div>"),
    text_input,

    # Section 4: Language selection
    widgets.HTML("<div class='section-header'>Select Language:</div>"),
    language,

    # Generate button
    widgets.HBox([generate_btn], layout=widgets.Layout(justify_content='center')),
    status,
    progress,

    widgets.HTML("</div>"),  # Close card

    # Results card
    widgets.HTML("<div class='card'>"),
    widgets.HTML("""
    <div style='text-align: center; margin-bottom: 15px;'>
      <h3 style='color: #6c5ce7; margin: 0;'>Your Voice Clone Results</h3>
      <p style='color: #636e72; margin-top: 5px;'>The enhanced audio will appear here</p>
    </div>
    """),
    output_audio,
    widgets.HTML("</div>")
], layout=widgets.Layout(width='850px', margin='0 auto', padding='30px'))

# Display the premium app interface
display(HTML("<div class='container'>"))
display(app)
display(HTML("</div>"))

# Add Font Awesome for icons
display(HTML("""
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
"""))

# Add initial JavaScript to hide recording status
display(HTML("""
<script>
document.querySelector('.recording-status').style.display = 'none';
</script>
"""))

 > tts_models/multilingual/multi-dataset/your_tts is already downloaded.
 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Model fully restored. 
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-